# Hinzufügen, Ändern und Löschen von Daten

Bei vielen Datensätzen möchtet ihr vielleicht eine Transformation basierend auf den Werten in einem Array, einer Serie oder einer Spalte in einem DataFrame durchführen. Hierfür betrachten wir die ersten Unicode-Zeichen:

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    {
        "Code": ["U+0000", "U+0001", "U+0002", "U+0003", "U+0004", "U+0005"],
        "Decimal": [0, 1, 2, 3, 4, 5],
        "Octal": ["001", "002", "003", "004", "004", "005"],
        "Key": ["NUL", "Ctrl-A", "Ctrl-B", "Ctrl-C", "Ctrl-D", "Ctrl-E"],
    }
)

df

,Code,Decimal,Octal,Key
0,U+0000,0,001,NUL
1,U+0001,1,002,Ctrl-A
2,U+0002,2,003,Ctrl-B
3,U+0003,3,004,Ctrl-C
4,U+0004,4,004,Ctrl-D
5,U+0005,5,005,Ctrl-E


## Daten hinzufügen

Angenommen, ihr möchtet eine Spalte hinzufügen, in der die Zeichen dem `C0`- oder `C1`-Steuercode zugewiesen werden:

In [3]:
control_code = {
    "u+0000": "C0",
    "u+0001": "C0",
    "u+0002": "C0",
    "u+0003": "C0",
    "u+0004": "C0",
    "u+0005": "C0",
}

Die `map`-Methode für eine Serie akzeptiert eine Funktion oder ein diktatähnliches Objekt, das eine Zuordnung enthält, aber hier haben wir ein kleines Problem, da einige die Codes in `control_code` kleingeschrieben sind, nicht jedoch in unserem DataFrame. Daher müssen wir jeden Wert mit der Methode `str.lower` in Kleinbuchstaben umwandeln:

In [4]:
lowercased = df["Code"].str.lower()

lowercased

0    u+0000
1    u+0001
2    u+0002
3    u+0003
4    u+0004
5    u+0005
Name: Code, dtype: object

In [5]:
df["Control code"] = lowercased.map(control_code)

df

,Code,Decimal,Octal,Key,Control code
0,U+0000,0,001,NUL,C0
1,U+0001,1,002,Ctrl-A,C0
2,U+0002,2,003,Ctrl-B,C0
3,U+0003,3,004,Ctrl-C,C0
4,U+0004,4,004,Ctrl-D,C0
5,U+0005,5,005,Ctrl-E,C0


Wir hätten auch eine Funktion übergeben können, die die ganze Arbeit erledigt:

In [6]:
df["Code"].map(lambda x: control_code[x.lower()])

0    C0
1    C0
2    C0
3    C0
4    C0
5    C0
Name: Code, dtype: object

Die Verwendung von `map` ist ein bequemer Weg, um elementweise Transformationen und andere Datenbereinigungsoperationen durchzuführen.

## Daten ändern

Mit der Methode [replace](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) lassen sich bestimmte Werte durch andere ersetzen.

In [7]:
s = pd.Series(["Manpower", "man-made", np.nan])

In [8]:
s.replace("Man", "Personal")

0    Manpower
1    man-made
2         NaN
dtype: object

In [9]:
s.replace("[Mm]an", "Personal", regex=True)

0    Personalpower
1    Personal-made
2              NaN
dtype: object

In [10]:
s.replace(["[Mm]an", np.nan], ["Personal", 0], regex=True)

0    Personalpower
1    Personal-made
2                0
dtype: object

In [11]:
s.replace(["[Mm]an", np.nan], ["Personal", len(s)], regex=True)

0    Personalpower
1    Personal-made
2                3
dtype: object

<div class="alert alert-block alert-info">

**Siehe auch:**

* [Verwalten fehlender Daten mit pandas](../../clean-prep/nulls.ipynb)
</div>

## Daten löschen

Einen oder mehrere Einträge aus einer Achse zu löschen ist einfach, wenn ihr bereits ein Index-Array oder eine Liste ohne diese Einträge habt.

Da dies ein wenig Mengenlehre erfordern kann, geben wir die Drop-Methode als neues Objekt ohne den oder die gelöschten Werten zurück:

In [12]:
s = pd.Series(np.random.randn(7))

s

0   -0.524343
1   -0.758496
2   -2.273494
3    1.907033
4    0.176254
5    0.407257
6   -2.370839
dtype: float64

In [13]:
new = s.drop(2)

new

0   -0.524343
1   -0.758496
3    1.907033
4    0.176254
5    0.407257
6   -2.370839
dtype: float64

In [14]:
new = s.drop([2, 3])

new

0   -0.524343
1   -0.758496
4    0.176254
5    0.407257
6   -2.370839
dtype: float64

Bei DataFrames können Indexwerte auf beiden Achsen gelöscht werden. Um dies zu veranschaulichen, erstellen wir zunächst einen Beispiel-DataFrame:

In [15]:
data = {
    "Code": ["U+0000", "U+0001", "U+0002", "U+0003", "U+0004", "U+0005"],
    "Decimal": [0, 1, 2, 3, 4, 5],
    "Octal": ["001", "002", "003", "004", "004", "005"],
    "Key": ["NUL", "Ctrl-A", "Ctrl-B", "Ctrl-C", "Ctrl-D", "Ctrl-E"],
}

df = pd.DataFrame(data)

df

,Code,Decimal,Octal,Key
0,U+0000,0,001,NUL
1,U+0001,1,002,Ctrl-A
2,U+0002,2,003,Ctrl-B
3,U+0003,3,004,Ctrl-C
4,U+0004,4,004,Ctrl-D
5,U+0005,5,005,Ctrl-E


In [16]:
df.drop([0, 1])

,Code,Decimal,Octal,Key
2,U+0002,2,003,Ctrl-B
3,U+0003,3,004,Ctrl-C
4,U+0004,4,004,Ctrl-D
5,U+0005,5,005,Ctrl-E


Ihr könnt auch Werte aus den Spalten entfernen, indem ihr `axis=1` oder `axis="columns"` übergebt:

In [17]:
df.drop("Decimal", axis=1)

,Code,Octal,Key
0,U+0000,001,NUL
1,U+0001,002,Ctrl-A
2,U+0002,003,Ctrl-B
3,U+0003,004,Ctrl-C
4,U+0004,004,Ctrl-D
5,U+0005,005,Ctrl-E


Viele Funktionen wie `drop`, die die Größe oder Form einer Reihe oder eines DataFrame ändern, können ein Objekt an Ort und Stelle manipulieren, ohne ein neues Objekt zurückzugeben:

In [18]:
df.drop(0, inplace=True)

df

,Code,Decimal,Octal,Key
1,U+0001,1,002,Ctrl-A
2,U+0002,2,003,Ctrl-B
3,U+0003,3,004,Ctrl-C
4,U+0004,4,004,Ctrl-D
5,U+0005,5,005,Ctrl-E


<div class="alert alert-block alert-warning">

**Warnung:**

Seid  vorsichtig mit der `inplace`-Funktion, da die Daten unwiderbringlich gelöscht werden.

<div class="alert alert-block alert-info">

**Siehe auch:**

* [Daten deduplizieren](../../clean-prep/deduplicate.ipynb)
</div>